<a href="https://colab.research.google.com/github/sanjayyy-22/hackathon-amcs/blob/main/Chatbot_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [3]:
!wget -q -O - ipv4.icanhazip.com

35.245.143.202


In [4]:
!pip install requests
!pip install PyPDF2
!pip install google-generativeai
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.5 MB/s eta 0:00:

In [5]:
%%writefile app.py
import streamlit as st
import requests
from PyPDF2 import PdfReader
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import google.generativeai as genai
import os
import glob

# Configure Google Gemini
genai.configure(api_key=os.getenv("api_key"))

def cleanup_old_files():
    # Delete all existing PDF files
    for file in glob.glob("*.pdf"):
        os.remove(file)

    # Clear ChromaDB collection by recreating it
    try:
        client.delete_collection("pdf_rag")
    except:
        pass
    collection = client.create_collection(name="pdf_rag", embedding_function=google_ef)
    return collection

# [Previous functions remain the same]
def download_pdf(url, save_path):
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, "wb") as file:
        file.write(response.content)

def load_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def split_text_recursively(text, max_length=1000, chunk_overlap=0):
    chunks = []
    start = 0
    text_length = len(text)
    while start < text_length:
        end = start + max_length
        if end < text_length:
            end = text.rfind(' ', start, end) + 1
            if end <= start:
                end = start + max_length
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - chunk_overlap
        if start >= text_length:
            break
    return chunks

# Initialize ChromaDB
google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.getenv("api_key"))
client = chromadb.PersistentClient(path="embeddings/gemini")

# Get or create initial collection
try:
    collection = client.get_collection(name="pdf_rag")
except:
    collection = client.create_collection(name="pdf_rag", embedding_function=google_ef)

# Streamlit UI
st.title("Personal PDF Q&A with Google Gemini")

# File upload with cleanup
uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])
if uploaded_file is not None:
    # Clear ChromaDB and get new collection
    collection = cleanup_old_files()

    with open("uploaded.pdf", "wb") as f:
        f.write(uploaded_file.read())
    st.success("PDF uploaded successfully!")

    pdf_text = load_pdf("uploaded.pdf")
    chunks = split_text_recursively(pdf_text, max_length=2000, chunk_overlap=200)

    for i, d in enumerate(chunks):
        collection.add(documents=[d], ids=[str(i)])
    st.success("PDF processed and added to the database!")

# User query
query = st.text_input("Enter your question:")
if query:
    def find_relevant_context(query, db, n_results=3):
        results = db.query(query_texts=[query], n_results=n_results)
        return [doc for doc in results['documents'][0]]

    context = find_relevant_context(query, collection)

    def create_prompt_for_gemini(query, context):
        prompt = f"""
        You are a helpful agent that answers questions using the text from the context below.
        Both the question and the context is shared with you and you should answer the
        question basis the context. If the context does not have enough information
        for you to answer the question correctly, inform about the absence of relevant
        context as part of your answer.

        Question : {query}
        \n
        Context : {' '.join(context)}
        \n
        Answer :
        """
        return prompt

    def generate_answer_from_gemini(prompt):
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        result = model.generate_content(prompt)
        return result.text

    prompt = create_prompt_for_gemini(query, context)
    answer = generate_answer_from_gemini(prompt)

    st.subheader("Answer:")
    st.write(answer)

Writing app.py


In [6]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [7]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://cruel-coins-sniff.loca.lt
